# **The Battle of Neighborhoods: Where to stay when you travel to Tokyo?**

### Final Assignment of Data Science Course by Coursera

Author : Miftachul Umam

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project, we will try to find an optimal location for stay on vacation. Specifically, this report will be targeted to low budget travellers or backpackers are interested in vacation in Tokyo Japan.

According to www.travelandleisure.com Tokyo is one of the most well-connected cities on the planet in terms of transportation. The best time to visit Tokyo is in the fall and spring, when temperatures are temperate and the scenery is stunning. Both cherry blossom season and autumn foliage season are excellent times to visit. Many festivals take place during the month of July, when Mount Fuji is also open for climbing.

Tokyo is a dizzying whirl of activity: you can practically feel it pulse, with the neon signs, the crush of people, and the perfectly punctual, high-speed trains whizzing by. Then there’s a pause, a bit of green and calm on the city’s temple grounds or classical gardens. 

Navigating the world’s largest metropolis—home to more than 13 million people—can be a daunting prospect for visitors. Tokyo’s maze of neighborhoods seems to offer up every imaginable sight and sound—some of them cacophonous and modern (speeding bullet trains; herds of hurrying, be-suited businessmen; bizarrely futuristic toilets), and some of them ancient (Buddhist shrines and temples; the waddling combat of sumo wrestlers). The trick here is to explore one enclave at a time; for instance, starting in Ginza or Shibuya for shopping, then heading to Shinjuku or Roppongi for nightlife.

Since there are lots of Hotel in Tokyo we will try to detect **the best price of hotels** based on http://insideairbnb.com/

We will use our data science powers to generate a few most promising neighborhoods based on these criteria. The advantages of each area will then be clearly expressed so that the best possible final location can be chosen by stakeholders. We got 102 Data of neighbourhoods wrom https://en.wikipedia.org/wiki/Category:Neighborhoods_of_Tokyo 

Using data science methodology and machine learning techniques like clustering, this project aims to provide solutions to answer the question: In the city of Tokyo, if a traveller is looking to stay on vacation, where would you recommend that they stay it?

## Data

Based on definition of our problem, factors that will influence our decission are:
* The list data of neighborhoods.
* Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map and also to get the venue data.
* Dataset of rating and price of existing hotel in the neighbourhoods. We will use this data to perform clustering on the neighbourhoods.
* Number of venues including tourists attractions, restaurants and shopping malls in the neighborhood.

Following data sources will be needed to extract/generate the required information:

I will get the list of neighbourhoods in Tokyo from Wikipedia. I have will use web scraping techniques to extract the data from the Wikipedia page, with the help of Python requests and pandas packages. Then we can get the latitude and longitude coordinates of the neighbourhoods using Python Geocoder package. After that, I will use the Foursquare API to get the venue data for those neighbourhoods.

AirBnB Inside will provide many categories of rating and price of existing hotel in the neighbourhoods and Foursquare API will provide many categories of the venue data, and we are particularly interested in the tourist attractions category in order to help us clustering the neighbourhoods. This project will make us use many of data science skills, from web scraping (Wikipedia), working with API (Foursquare), data cleaning, data wrangling, to machine learning (K-means clustering) and map visualization (Folium).

## Methodology

* Perform scraping using Python requests and beautifulsoup packages to extract the list of neighbourhoods data.
* Download data inside from AirBnB for geting the data of accommodations, Then, we will randomly choose 1000 accommodations. In this case i got other data of neighbourhoods name by airbnb that totally different with the wikipedia, then I consider use AirBnB neighbourhood name becasue I have struggled to grouping based on haversine formula to merging the wikipedia neighbourhood data and AirBnB data. I don't clean the data of neighbourhood based on wikipedia data for ideation the next project. 
* After that, we will use Foursquare API to obtain information on venues nearby to our top 1000 accommodations. In our final step, we will cluster our accommodations with k-means clustering and provide recommendations to travellers.
* Using The Foursquare API allows application developers to interact with the Foursquare platform. The API itself is a RESTful set of addresses to which you can send requests, so there’s really nothing to download onto your server. 


## Analysis

#### Data Preparation

Install all of required packages 

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install geopy
!pip install geocoder
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 122kB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 3.5MB/s eta 0:00:01████████▊  | 5.2MB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:011
     |████████████████████████████████| 829kB 9.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment locati

#### Scraping data of neighbourhood

In [2]:
# Send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_of_Tokyo").text
# Parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# Create a list to store neighbourhood data
neighborhoodList = []
# Append the data into the list
for row in soup.find_all("div", id="mw-pages")[0].findAll("li"):
  neighborhoodList.append(row.text)
# Create a new DataFrame from the list
Tokyo_df = pd.DataFrame({"Neighborhood": neighborhoodList})
Tokyo_df


,Neighborhood
0,Agariyashiki
1,Akihabara
2,"Aoyama, Tokyo"
3,"Arai, Tokyo"
4,Asagaya
...,...
97,Yotsuya
98,Yoyogi
99,Yoyogikamizonochō
100,Yūrei zaka


In [3]:
import geocoder
# Defining a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Tokyo, Japan'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Tokyo_df["Neighborhood"].tolist()]
coords

[[35.72646208300006, 139.7051560540001],
 [35.702170840000065, 139.7744091400001],
 [35.68945633000004, 139.6917160900001],
 [35.68945633000004, 139.6917160900001],
 [35.704890000000034, 139.6362600000001],
 [35.69197253400006, 139.7414457110001],
 [35.69959896000006, 139.67506014000003],
 [35.65076536000004, 139.70468298000003],
 [35.658362440000076, 139.69791276],
 [35.64532595000003, 139.7165050200001],
 [35.64466222000004, 139.70816845000002],
 [35.64845099000007, 139.70606734000012],
 [35.689780760000076, 139.7340734400001],
 [35.651806760000056, 139.69809175000012],
 [35.68945633000004, 139.6917160900001],
 [35.67771698000007, 139.70633101900012],
 [35.67795733000003, 139.6858345500001],
 [35.674993474000075, 139.75956009000004],
 [35.65380213000003, 139.71012802000007],
 [35.748591660000045, 139.58732147000012],
 [35.652052350000076, 139.7175139100001],
 [35.69257497000007, 139.75741702000005],
 [35.75654117000005, 139.46953437000002],
 [35.66186617000005, 139.32580599000005],
 

In [6]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
Tokyo_df['Latitude'] = df_coords['Latitude']
Tokyo_df['Longitude'] = df_coords['Longitude']
print(Tokyo_df.shape)
Tokyo_df.head()

(102, 3)


,Neighborhood,Latitude,Longitude
0,Agariyashiki,35.726462,139.705156
1,Akihabara,35.702171,139.774409
2,"Aoyama, Tokyo",35.689456,139.691716
3,"Arai, Tokyo",35.689456,139.691716
4,Asagaya,35.704890,139.636260


In [7]:
# Getting the coordinates of Tokyo
address = 'Tokyo, Japan'
geolocator = Nominatim(user_agent="umam")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo, Japan {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo, Japan 35.6828387, 139.7594549.


In [9]:
map_Tokyo_df = folium.Map(location=[latitude, longitude], zoom_start=12)
# Adding markers to map
for lat, lng, neighborhood in zip(Tokyo_df['Latitude'],  Tokyo_df['Longitude'], Tokyo_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='red',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_Tokyo_df)
map_Tokyo_df

### Download List Hotel from AirBnB

### we can get the AirBnB Inside Data from http://insideairbnb.com/

#### Exploratory AirBnB Data 

In [10]:
df_hotel=open('listings.csv') 
df_hotel = pd.read_csv(df_hotel)
df_hotel.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71721,139.82596,Entire home/apt,11000,3,165,2020-03-04,1.49,1,340
1,776070,Kero-kero house room 1,801494,Kotaro Kei And Litte Miya,NaN,Kita Ku,35.73844,139.76917,Private room,7341,3,228,2020-01-18,2.37,1,213
2,899003,"Classy room @Shinjuku, Takadanoba",4799233,Yu,NaN,Shinjuku Ku,35.70865,139.69681,Entire home/apt,5200,30,93,2019-11-25,0.99,2,180
3,905944,4F - Near Shinjuku & Shibuya w/Free WiFi,4847803,Best Stay In Tokyo!,NaN,Shibuya Ku,35.67878,139.67847,Entire home/apt,15731,1,185,2020-03-27,2.01,6,330
4,1016831,WOMAN ONLY LICENSED ! Cosy & Cat behnd Shibuya,5596383,Wakana,NaN,Setagaya Ku,35.65833,139.67153,Private room,15214,1,211,2020-03-16,2.29,1,139


In [11]:
Tokyo_Hotel = df_hotel.drop(['id','host_id','host_name','neighbourhood_group','room_type','minimum_nights','number_of_reviews','last_review','reviews_per_month','calculated_host_listings_count','availability_365'],1)
Tokyo_Hotel.head()
Tokyo_Hotel.count

<bound method DataFrame.count of                                                   name neighbourhood  \
0                            Oshiage Holiday Apartment     Sumida Ku   
1                               Kero-kero house room 1       Kita Ku   
2                    Classy room @Shinjuku, Takadanoba   Shinjuku Ku   
3             4F - Near Shinjuku & Shibuya w/Free WiFi    Shibuya Ku   
4       WOMAN ONLY LICENSED ! Cosy & Cat behnd Shibuya   Setagaya Ku   
...                                                ...           ...   
11710                            C3_池袋站6分钟两卧室三张双人床便利公寓    Toshima Ku   
11711    Studio with loft, Skytree/Asakusa 10mins walk     Sumida Ku   
11712   15mins to Shinjuku/Ikebukuro/Ginza/Tokyo! 2ppl   Shinjuku Ku   
11713                                           民宿：楓晩亭      Tama Shi   
11714  Chill Inn 方南～2LDKデザイナーズ駐車場付き一軒家/3駅アクセス/Wi-Fi完備～   Suginami Ku   

       latitude  longitude  price  
0      35.71721  139.82596  11000  
1      35.73844  139.76917   7

### We also got data of neighbourhood name in AirBnB data

In [12]:
Neighbourhood_airbnb=Tokyo_Hotel['neighbourhood'].unique()
Neighbourhood_airbnb

array(['Sumida Ku', 'Kita Ku', 'Shinjuku Ku', 'Shibuya Ku', 'Setagaya Ku',
       'Adachi Ku', 'Katsushika Ku', 'Meguro Ku', 'Toshima Ku', 'Koto Ku',
       'Shinagawa Ku', 'Minato Ku', 'Suginami Ku', 'Akishima Shi',
       'Bunkyo Ku', 'Taito Ku', 'Itabashi Ku', 'Edogawa Ku', 'Arakawa Ku',
       'Machida Shi', 'Higashimurayama Shi', 'Chuo Ku', 'Nerima Ku',
       'Kokubunji Shi', 'Hino Shi', 'Mitaka Shi', 'Nakano Ku',
       'Hachioji Shi', 'Musashino Shi', 'Koganei Shi', 'Ota Ku',
       'Akiruno Shi', 'Chiyoda Ku', 'Kodaira Shi', 'Fuchu Shi',
       'Fussa Shi', 'Tama Shi', 'Komae Shi', 'Chofu Shi',
       'Nishitokyo Shi', 'Hinohara Mura', 'Ome Shi', 'Kunitachi Shi',
       'Higashiyamato Shi', 'Okutama Machi', 'Hamura Shi',
       'Tachikawa Shi', 'Inagi Shi', 'Musashimurayama Shi'], dtype=object)

In [13]:
Neighbourhood_airbnb = pd.DataFrame(Neighbourhood_airbnb, columns=['neighbourhood'])
Neighbourhood_airbnb

,neighbourhood
0,Sumida Ku
1,Kita Ku
2,Shinjuku Ku
3,Shibuya Ku
4,Setagaya Ku
5,Adachi Ku
6,Katsushika Ku
7,Meguro Ku
8,Toshima Ku
9,Koto Ku


### Short the data of neighbourhood_airbnb and compare with wikipedia's data 

In [16]:
Neighbourhood_airbnb.sort_values('neighbourhood').head(10)


,neighbourhood
5,Adachi Ku
31,Akiruno Shi
13,Akishima Shi
18,Arakawa Ku
14,Bunkyo Ku
32,Chiyoda Ku
38,Chofu Shi
21,Chuo Ku
17,Edogawa Ku
34,Fuchu Shi


In [19]:
Tokyo_df.head(10)

,Neighborhood,Latitude,Longitude
0,Agariyashiki,35.726462,139.705156
1,Akihabara,35.702171,139.774409
2,"Aoyama, Tokyo",35.689456,139.691716
3,"Arai, Tokyo",35.689456,139.691716
4,Asagaya,35.704890,139.636260
5,Banchō,35.691973,139.741446
6,"Chūō, Nakano, Tokyo",35.699599,139.675060
7,"Daikanyamachō, Shibuya",35.650765,139.704683
8,Dōgenzaka (district),35.658362,139.697913
9,"Ebisu, Shibuya",35.645326,139.716505


### With manually comparation, the result is the list names of neighbourhood based airbnb data's is totally different with wikipedia's. I tried to using haversine formula to merging the two data, but I have struggled then I consider choose neighbourhood data by Airbnb for simple way. Generally speaking, people can search this location in google maps when they want to know the location. 

In [20]:
import geocoder
# Defining a function to get coordinates
def get_latlng(neighbourhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Tokyo, Japan'.format(neighbourhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
Coord = [ get_latlng(neighborhood) for neighborhood in Neighbourhood_airbnb["neighbourhood"].tolist()]
Coord

[[35.710706680000044, 139.80154019000008],
 [35.75283897000003, 139.7335186900001],
 [35.693797560000064, 139.7034401300001],
 [35.663686840000025, 139.6977910600001],
 [35.64654360000003, 139.65322156000002],
 [35.77481106000005, 139.80453694000005],
 [35.74345381000006, 139.84722882000005],
 [35.64148995000005, 139.69827334000001],
 [35.726128190000054, 139.7166933300001],
 [35.67282283000003, 139.81728987000008],
 [35.609160400000064, 139.7301606100001],
 [35.658017080000036, 139.7515459000001],
 [35.699731420000035, 139.63613778],
 [35.70569591000003, 139.35363336000012],
 [35.707595010000034, 139.7522101500001],
 [35.71259536000008, 139.77998619000005],
 [35.751074430000074, 139.70919442000002],
 [35.707013860000075, 139.86836701000004],
 [35.73609347000007, 139.78340269000012],
 [35.54662537000007, 139.43869566],
 [35.754610350000064, 139.46846855],
 [35.67057158000006, 139.77198757000008],
 [35.73569997000004, 139.65160688000003],
 [35.71039220000006, 139.4632071000001],
 [35.67

In [162]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coord = pd.DataFrame(Coord, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
Neighbourhood_airbnb['Latitude'] = df_coord['Latitude']
Neighbourhood_airbnb['Longitude'] = df_coord['Longitude']
print(Neighbourhood_airbnb.shape)
Neighbourhood_airbnb
Tokyo_neigh = Neighbourhood_airbnb
Tokyo_neigh.head()

(49, 3)


,neighbourhood,Latitude,Longitude
0,Sumida Ku,35.710707,139.801540
1,Kita Ku,35.752839,139.733519
2,Shinjuku Ku,35.693798,139.703440
3,Shibuya Ku,35.663687,139.697791
4,Setagaya Ku,35.646544,139.653222


In [22]:
map_Tokyo_neigh = folium.Map(location=[latitude, longitude], zoom_start=12)
# Adding markers to map
for lat, lng, neighbourhood in zip(Tokyo_neigh['Latitude'],  Tokyo_neigh['Longitude'], Tokyo_neigh['neighbourhood']):
 label = '{}'.format(neighbourhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_Tokyo_neigh)
map_Tokyo_neigh

## Clustering the hotel in the neighbourhood by price

### Cheapest 1000 Hotels 

In [59]:
Hotel_cheapest=Tokyo_Hotel.sort_values('price').head(1000)
Hotel_cheapest

,name,neighbourhood,latitude,longitude,price
1969,Poket Wi-Fi 509 GK,Sumida Ku,35.69405,139.81376,737
1958,Poket Wi-Fi 402 GK,Sumida Ku,35.69506,139.81293,810
1966,Poket Wi-Fi 506 GK,Sumida Ku,35.69485,139.81236,840
1961,Poket Wi-Fi 501 GK,Sumida Ku,35.69388,139.81217,880
1963,Poket Wi-Fi 503 GK,Sumida Ku,35.69540,139.81297,880
...,...,...,...,...,...
8685,上野附近温馨独立公寓101室Ueno Nearby Cozy Apartment 101,Taito Ku,35.72538,139.78261,2600
8103,Koenji Station walk 3,Suginami Ku,35.70220,139.64919,2600
8101,3 minutes on foot from Koenji,Suginami Ku,35.70395,139.64907,2600
2627,"[C3]near to Shinjuku,Shibuya,Ueno etc,24H checkIn",Kita Ku,35.73874,139.74878,2600


In [60]:
Tokyo_hotel_price = Hotel_cheapest.drop(['latitude','longitude','name','neighbourhood'],1)
Tokyo_hotel_price.head()

,price
1969,737
1958,810
1966,840
1961,880
1963,880


In [61]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(Tokyo_hotel_price)
labels = k_means.labels_

print(labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [65]:
Hotel_cheapest["Labels"] = labels
Hotel_cheapest.head(10)

,name,neighbourhood,latitude,longitude,price,Labels
1969,Poket Wi-Fi 509 GK,Sumida Ku,35.69405,139.81376,737,1
1958,Poket Wi-Fi 402 GK,Sumida Ku,35.69506,139.81293,810,1
1966,Poket Wi-Fi 506 GK,Sumida Ku,35.69485,139.81236,840,1
1961,Poket Wi-Fi 501 GK,Sumida Ku,35.69388,139.81217,880,1
1963,Poket Wi-Fi 503 GK,Sumida Ku,35.69540,139.81297,880,1
1962,Poket Wi-Fi 502 GK,Sumida Ku,35.69383,139.81346,910,1
1989,Poket Wi-Fi 401 GK,Sumida Ku,35.69493,139.81418,980,1
10865,Enjoy your private time ｰ Nest Inn Tabata A -,Arakawa Ku,35.73542,139.76562,1071,1
10351,Enjoy your private time - Nest Inn Tabata B -,Kita Ku,35.73620,139.76439,1082,1
1765,8 mins to Station/private room with TV★S3-3,Adachi Ku,35.80921,139.76721,1100,1


In [63]:
Hotel_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Hotel_cheapest['latitude'], Hotel_cheapest['longitude'], Hotel_cheapest['price'], Hotel_cheapest['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Hotel_clusters)
       
Hotel_clusters

## API FOURSQUARE FOR VENUES

In [96]:
CLIENT_ID = 'MK5XM5OYBCD2FBPUZSGQE2TETCYW1C25HSLZ3FDCFFCLZSO2' # your Foursquare ID
CLIENT_SECRET = 'P340H0WQIXAN011QUXKEBTH0PQ0ZMFVGW0RVE3ATNFIDQ1NR' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MK5XM5OYBCD2FBPUZSGQE2TETCYW1C25HSLZ3FDCFFCLZSO2
CLIENT_SECRET:P340H0WQIXAN011QUXKEBTH0PQ0ZMFVGW0RVE3ATNFIDQ1NR


### Searcing venues in the neighbourhood

In [97]:
def getNearbyVenues(names, latitudes, longitudes, radius=200, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [98]:
# type your answer here
Tokyo_venues = getNearbyVenues(names=Hotel_cheapest['neighbourhood'],
                                   latitudes=Hotel_cheapest['latitude'],
                                   longitudes=Hotel_cheapest['longitude']
                                  )


Sumida Ku
Sumida Ku
Sumida Ku
Sumida Ku
Sumida Ku
Sumida Ku
Sumida Ku
Arakawa Ku
Kita Ku
Adachi Ku
Katsushika Ku
Sumida Ku
Sumida Ku
Sumida Ku
Edogawa Ku
Adachi Ku
Sumida Ku
Katsushika Ku
Sumida Ku
Sumida Ku
Edogawa Ku
Sumida Ku
Adachi Ku
Sumida Ku
Adachi Ku
Sumida Ku
Sumida Ku
Sumida Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Sumida Ku
Katsushika Ku
Adachi Ku
Sumida Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Adachi Ku
Sumida Ku
Sumida Ku
Adachi Ku
Sumida Ku
Adachi Ku
Sumida Ku
Adachi Ku
Adachi Ku
Sumida Ku
Adachi Ku
Adachi Ku
Adachi Ku
Katsushika Ku
Katsushika Ku
Katsushika Ku
Katsushika Ku
Katsushika Ku
Sumida Ku
Sumida Ku
Ota Ku
Taito Ku
Toshima Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Shinjuku Ku
Sumida Ku
Adachi Ku
Suginami K

In [99]:
print(Tokyo_venues.shape)
Tokyo_venues.head()

(6847, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sumida Ku,35.69405,139.81376,サウナ&カプセル ニューウイング,35.694132,139.812702,Sauna / Steam Room
1,Sumida Ku,35.69405,139.81376,Bariki (馬力),35.694051,139.813272,Sake Bar
2,Sumida Ku,35.69405,139.81376,Asia Curry House (アジアカレーハウス),35.694550,139.813731,Bangladeshi Restaurant
3,Sumida Ku,35.69405,139.81376,Moxy Tokyo Kinshicho (モクシー東京錦糸町),35.693448,139.813476,Hotel
4,Sumida Ku,35.69405,139.81376,Madai Ramen Mengyo (真鯛らーめん 麺魚),35.694200,139.811900,Ramen Restaurant


In [100]:
Tokyo_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adachi Ku,259,259,259,259,259,259
Arakawa Ku,259,259,259,259,259,259
Bunkyo Ku,222,222,222,222,222,222
Chiyoda Ku,30,30,30,30,30,30
Chuo Ku,314,314,314,314,314,314
Edogawa Ku,123,123,123,123,123,123
Hino Shi,15,15,15,15,15,15
Inagi Shi,7,7,7,7,7,7
Itabashi Ku,389,389,389,389,389,389


In [147]:
print('There are {} uniques categories.'.format(len(Tokyo_venues['Venue Category'].unique())))

There are 245 uniques categories.


## Analize Venues in Neighbourhood

In [148]:
Tokyo_grouped = Tokyo_onehot.groupby('Neighborhood').sum().reset_index()
Tokyo_grouped

,Neighborhood,ATM,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Adachi Ku,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0
1,Arakawa Ku,3,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,12,9,0,0
2,Bunkyo Ku,5,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
3,Chiyoda Ku,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
4,Chuo Ku,0,0,0,0,0,0,0,0,0,...,0,7,0,0,0,0,0,7,0,0
5,Edogawa Ku,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Hino Shi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
7,Inagi Shi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Itabashi Ku,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,2,0
9,Katsushika Ku,2,0,0,0,0,0,0,0,5,...,0,1,0,0,0,0,0,0,0,0


In [150]:
num_top_venues = 5

for hood in Tokyo_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =Tokyo_grouped[Tokyo_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(int)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adachi Ku----
               venue  freq
0  Convenience Store    42
1       Intersection    19
2      Deli / Bodega    14
3   Ramen Restaurant    11
4        Supermarket    11


----Arakawa Ku----
                 venue  freq
0    Convenience Store    63
1                 Café    15
2  Yakitori Restaurant    12
3   Chinese Restaurant    12
4             Sake Bar    11


----Bunkyo Ku----
                venue  freq
0   Convenience Store    23
1                Café    22
2               Hotel    19
3    Ramen Restaurant    17
4  Italian Restaurant    16


----Chiyoda Ku----
                    venue  freq
0                Sake Bar     6
1       Indian Restaurant     2
2      Yoshoku Restaurant     2
3     Sporting Goods Shop     2
4  Singaporean Restaurant     2


----Chuo Ku----
                  venue  freq
0      Sushi Restaurant    58
1  Monjayaki Restaurant    24
2                 Hotel    17
3          Noodle House    16
4    Italian Restaurant    15


----Edogawa Ku----
     

In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [199]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Tokyo_grouped['Neighborhood']

for ind in np.arange(Tokyo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tokyo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adachi Ku,Convenience Store,Intersection,Deli / Bodega,Supermarket,Ramen Restaurant,Japanese Restaurant,Park,Soba Restaurant,Candy Store,Bistro
1,Arakawa Ku,Convenience Store,Café,Yakitori Restaurant,Chinese Restaurant,Sake Bar,Japanese Restaurant,Grocery Store,Yoshoku Restaurant,Candy Store,Ramen Restaurant
2,Bunkyo Ku,Convenience Store,Café,Hotel,Ramen Restaurant,Italian Restaurant,BBQ Joint,Coffee Shop,Dessert Shop,Tempura Restaurant,Indian Restaurant
3,Chiyoda Ku,Sake Bar,Indian Restaurant,Sporting Goods Shop,Soba Restaurant,Ski Shop,Singaporean Restaurant,Outdoor Supply Store,Yoshoku Restaurant,Ramen Restaurant,Udon Restaurant
4,Chuo Ku,Sushi Restaurant,Monjayaki Restaurant,Hotel,Noodle House,Italian Restaurant,BBQ Joint,Sake Bar,Chinese Restaurant,Soba Restaurant,Japanese Restaurant


## Cluster Neighborhoods

In [195]:
from sklearn.preprocessing import StandardScaler
Tokyo_grouped_clustering = Tokyo_grouped.drop('Neighborhood', 1)
X = Tokyo_grouped_clustering.values[:,1:]
X = np.nan_to_num(X)
Tokyo_grouped_clustering = StandardScaler().fit_transform(X)
Tokyo_grouped_clustering

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.19245009, -0.19245009, -0.26211122, ..., -0.59132948,
        -0.19245009, -0.19245009],
       [-0.19245009, -0.19245009, -0.26211122, ...,  2.1180347 ,
        -0.19245009, -0.19245009],
       [-0.19245009, -0.19245009, -0.26211122, ..., -0.29028902,
        -0.19245009, -0.19245009],
       ...,
       [ 5.19615242, -0.19245009,  2.18426014, ...,  2.1180347 ,
        -0.19245009, -0.19245009],
       [-0.19245009, -0.19245009, -0.26211122, ...,  1.51595377,
        -0.19245009, -0.19245009],
       [-0.19245009, -0.19245009, -0.26211122, ..., -0.59132948,
        -0.19245009, -0.19245009]])

In [202]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tokyo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 2, 1, 0], dtype=int32)

In [200]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tokyo_merged = Tokyo_neigh

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Tokyo_merged = Tokyo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighbourhood')

Tokyo_merged.head() # check the last columns!

,neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sumida Ku,35.710707,139.801540,2.0,Convenience Store,Sake Bar,Ramen Restaurant,Japanese Restaurant,Nabe Restaurant,Intersection,Hostel,Soba Restaurant,Bus Stop,Hotel
1,Kita Ku,35.752839,139.733519,0.0,Convenience Store,Ramen Restaurant,Sake Bar,Chinese Restaurant,Japanese Restaurant,Playground,Park,Grocery Store,Café,Tempura Restaurant
2,Shinjuku Ku,35.693798,139.703440,3.0,Korean Restaurant,Convenience Store,BBQ Joint,Hotel,Sake Bar,Ramen Restaurant,Supermarket,Italian Restaurant,Discount Store,Chinese Restaurant
3,Shibuya Ku,35.663687,139.697791,0.0,Convenience Store,Ramen Restaurant,BBQ Joint,Chinese Restaurant,Sake Bar,Italian Restaurant,Playground,Indian Restaurant,Grocery Store,Japanese Curry Restaurant
4,Setagaya Ku,35.646544,139.653222,0.0,Playground,Ramen Restaurant,Dessert Shop,Convenience Store,Yakitori Restaurant,Sake Bar,Kushikatsu Restaurant,Fish Market,Bakery,Intersection


In [201]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tokyo_merged['Latitude'], Tokyo_merged['Longitude'], Tokyo_merged['neighbourhood'], Tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

## **Examine Clusters**

### Cluster 1-5

In [203]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 0, Tokyo_merged.columns[[0] + list(range(4, Tokyo_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Kita Ku,Convenience Store,Ramen Restaurant,Sake Bar,Chinese Restaurant,Japanese Restaurant,Playground,Park,Grocery Store,Café,Tempura Restaurant
3,Shibuya Ku,Convenience Store,Ramen Restaurant,BBQ Joint,Chinese Restaurant,Sake Bar,Italian Restaurant,Playground,Indian Restaurant,Grocery Store,Japanese Curry Restaurant
4,Setagaya Ku,Playground,Ramen Restaurant,Dessert Shop,Convenience Store,Yakitori Restaurant,Sake Bar,Kushikatsu Restaurant,Fish Market,Bakery,Intersection
5,Adachi Ku,Convenience Store,Intersection,Deli / Bodega,Supermarket,Ramen Restaurant,Japanese Restaurant,Park,Soba Restaurant,Candy Store,Bistro
6,Katsushika Ku,Convenience Store,Intersection,Park,Train Station,Platform,Japanese Restaurant,Sake Bar,Unagi Restaurant,Bakery,Grocery Store
7,Meguro Ku,Playground,Concert Hall,Noodle House,Chinese Restaurant,Intersection,Coffee Shop,Tonkatsu Restaurant,Furniture / Home Store,Bus Station,Dessert Shop
8,Toshima Ku,Convenience Store,Historic Site,Chinese Restaurant,Bus Stop,Café,Grocery Store,Japanese Restaurant,Tonkatsu Restaurant,Park,Indian Restaurant
10,Shinagawa Ku,Park,Bus Stop,Unagi Restaurant,Japanese Restaurant,Pharmacy,Tempura Restaurant,Shopping Mall,Asian Restaurant,Go Kart Track,Café
11,Minato Ku,Japanese Restaurant,Sake Bar,BBQ Joint,Italian Restaurant,Seafood Restaurant,Yoshoku Restaurant,Udon Restaurant,Sushi Restaurant,Donburi Restaurant,Nabe Restaurant
12,Suginami Ku,Convenience Store,Supermarket,ATM,Indian Restaurant,Café,Park,Train Station,Discount Store,Intersection,Sake Bar


In [204]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 1, Tokyo_merged.columns[[0] + list(range(4, Tokyo_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Taito Ku,Convenience Store,Ramen Restaurant,Café,Grocery Store,Tonkatsu Restaurant,Hostel,Bath House,Japanese Restaurant,Chinese Restaurant,Sake Bar


In [205]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 2, Tokyo_merged.columns[[0] + list(range(4, Tokyo_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sumida Ku,Convenience Store,Sake Bar,Ramen Restaurant,Japanese Restaurant,Nabe Restaurant,Intersection,Hostel,Soba Restaurant,Bus Stop,Hotel


In [206]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 3, Tokyo_merged.columns[[0] + list(range(4, Tokyo_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Shinjuku Ku,Korean Restaurant,Convenience Store,BBQ Joint,Hotel,Sake Bar,Ramen Restaurant,Supermarket,Italian Restaurant,Discount Store,Chinese Restaurant


In [207]:
Tokyo_merged.loc[Tokyo_merged['Cluster Labels'] == 4, Tokyo_merged.columns[[0] + list(range(4, Tokyo_merged.shape[1]))]]

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Koto Ku,Convenience Store,BBQ Joint,Ramen Restaurant,Sake Bar,Park,Italian Restaurant,Intersection,Café,Chinese Restaurant,Bus Stop


## RESULT

Neighbourhood
Best on data the best neigbourhood for stay is Sumida Ku,Sumida is in the northeastern part of the mainland portion of Tokyo. The Sumida and Arakawa are the major rivers, and form parts of its boundaries. 

Landmarks Sumida:
*Tokyo Skytree: A digital terrestrial television broadcasting tower used by NHK and other broadcasters. It is the tallest tower in the world and the tallest man-made structure in Japan.\
*Ryōgoku Kokugikan (National Sumo Stadium)\
*Edo-Tokyo Museum\
*Asahi Breweries Headquarters: The Asahi Beer Hall with the Asahi flame created by French designer Philippe Starck in 1989, is one of Tokyo's most recognizable modern structures.\
*Eko-in: Buddhist temple\
*Honjo Matsuzaka-cho Park: the residence of Kira Yoshinaka stood on this site. The Forty-seven rōnin took his life during the Genroku era.\
*Hokusai-dori (street), with a series of prints by famed Japanese artist Hokusai who was born in the Kamezawa area of Sumida.\
*Sumida Triphony Hall, concert hall\
*Tobu Museum\
*Tokyo Irei-do (Tokyo Memorial Hall): a memorial to those unidentified people who died in the Great Kantō earthquake, the Bombing of Tokyo in World War II and other catastrophes; by Itō Chūta\
*Yokoamicho Park, in the Yokoami district\


Based on foursquare the favorite venues in sumida is Convenience Store, Sake Bar, Ramen Restaurant, Japan Resto, Bus Stop and Hotel. This is strengthen the hypothesis that Sumida is the right destination to stay on vacation